# Fetch data 

In [5]:
import requests
import json 
import pandas as pd
import numpy as np
def read_key(dir):
    f = open(dir,"r")
    return f.readline()[:-1]

def fetch_Data(url , key=read_key('../.key_subscription')):
    
    headers = {"Ocp-Apim-Subscription-Key": key,
            'Content-Type': 'application/json'}
    res = requests.get(url, headers=headers)


    response = json.loads(res.text)
    return response

## News_by_date

In [69]:
url = "https://api.sportsdata.io/v3/nhl/scores/json/NewsByDate/2023-11-01"
data_2 = fetch_Data(url)
data_2

[{'NewsID': 27140,
  'PlayerID': 30003147,
  'TeamID': 1,
  'Team': 'BOS',
  'Title': 'Charlie McAvoy Suspended Four Games',
  'Content': "Boston Bruins defenseman Charlie McAvoy has received a four-game suspension for an illegal check to the head of Florida Panthers defenseman Oliver Ekman-Larsson. The incident resulted in McAvoy getting a match penalty in Monday's game, and he'll now have plenty of time to think about his actions. McAvoy scored in a second consecutive game Monday and has been on fire with eight points in nine games. The Bruins also lost Matt Grzelcyk (upper body) to injury in Monday's game, meaning the entire first defensive pairing got wiped out in one night. Hampus Lindholm and Brandon Carlo should receive a nice boost in playing time now, while the rest of the blue line will need to put in extra shifts as well. McAvoy will be eligible to return to action on Nov. 11 against Montreal.",
  'Url': 'https://www.rotoballer.com/player-news/charlie-mcavoy-suspended-four-g

In [3]:
df_2 = pd.DataFrame(data_2)
df_2.columns

Index(['NewsID', 'PlayerID', 'TeamID', 'Team', 'Title', 'Content', 'Url',
       'Source', 'TermsOfUse', 'Updated'],
      dtype='object')

In [4]:
df_2["Updated"].value_counts(dropna=False
                             )

2023-11-01T14:41:02    1
Name: Updated, dtype: int64

## Make_dataset

In [6]:
from datetime import datetime , timedelta
now = datetime.now() 
now.strftime("%Y/%m/%d") , now

('2023/11/17', datetime.datetime(2023, 11, 17, 5, 19, 18, 210081))

In [7]:
# Define the columns that we want from GameByDate , TeamGameStatByDate , NewsByDate
Game_columns = ["GameID","DateTime","IsClosed","AwayTeamID","HomeTeamID","AwayTeamScore","HomeTeamScore"]
Stat_columns = ["StatID","TeamID","Name","FantasyPoints","FantasyPointsFanDuel","FantasyPointsDraftKings","FantasyPointsYahoo",
    "Minutes",
    "Seconds",
    "Goals",
    "Assists",
    "ShotsOnGoal",
    "PowerPlayGoals",
    "ShortHandedGoals",
    "EmptyNetGoals",
    "PowerPlayAssists",
    "ShortHandedAssists",
    "HatTricks",
    "ShootoutGoals",
    "PlusMinus",
    "PenaltyMinutes",
    "Blocks",
    "Hits",
    "Takeaways",
    "Giveaways",
    "FaceoffsWon",
    "FaceoffsLost",
    "Shifts",
    "GoaltendingMinutes",
    "GoaltendingSeconds",
    "GoaltendingShotsAgainst",
    "GoaltendingGoalsAgainst",
    "GoaltendingSaves",
    "GoaltendingWins",
    "GoaltendingLosses",
    "GoaltendingShutouts",
    "GoaltendingOvertimeLosses",
    "FantasyPointsFantasyDraft"]

News_columns = ["TeamID" , "NewsID" , "Content" , "DateTime" ,"Url","Source"]

In [8]:
df_Game = []
df_news = []
df_Stat = []
days = 7
for i in range(1,days):
    date_Game = (now - timedelta(days=i)).strftime("%Y-%m-%d")
    date_Stat = (now - timedelta(days=i)).strftime("%Y-%m-%d")
    date_News = (now - timedelta(days=i+1)).strftime("%Y-%m-%d")
    # Fetch Games
    url = f"https://api.sportsdata.io/v3/nhl/scores/json/GamesByDate/{date_Game}"
    data_Game = fetch_Data(url)
    # Fetch Stats
    url = f"https://api.sportsdata.io/v3/nhl/scores/json/TeamGameStatsByDate/{date_Stat}"
    data_Stat = fetch_Data(url)
    # Fetch News
    url = f"https://api.sportsdata.io/v3/nhl/scores/json/NewsByDate/{date_News}"
    data_News = fetch_Data(url)
    df_Game.append(pd.DataFrame(data_Game)[Game_columns])
    df_Stat.append(pd.DataFrame(data_Stat)[Stat_columns])
    for dic in data_News : # add the date to the data
        dic["DateTime"] = datetime.strptime(date_News, '%Y-%m-%d')
    df_news.append(pd.DataFrame(data_News)[News_columns])

df_Game = pd.concat(df_Game,ignore_index=True)
df_news = pd.concat(df_news,ignore_index=True)
df_Stat = pd.concat(df_Stat,ignore_index=True)

df_Game["DateTime"] = df_Game["DateTime"].apply(lambda x : datetime.strptime(x.split("T")[0], '%Y-%m-%d'))

In [9]:
df_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   TeamID    6 non-null      int64         
 1   NewsID    6 non-null      int64         
 2   Content   6 non-null      object        
 3   DateTime  6 non-null      datetime64[ns]
 4   Url       6 non-null      object        
 5   Source    6 non-null      object        
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 416.0+ bytes


In [10]:
df_Game.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   GameID         41 non-null     int64         
 1   DateTime       41 non-null     datetime64[ns]
 2   IsClosed       41 non-null     bool          
 3   AwayTeamID     41 non-null     int64         
 4   HomeTeamID     41 non-null     int64         
 5   AwayTeamScore  37 non-null     float64       
 6   HomeTeamScore  37 non-null     float64       
dtypes: bool(1), datetime64[ns](1), float64(2), int64(3)
memory usage: 2.1 KB


In [13]:
df_Stat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 38 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   StatID                     82 non-null     int64  
 1   TeamID                     82 non-null     int64  
 2   Name                       82 non-null     object 
 3   FantasyPoints              82 non-null     float64
 4   FantasyPointsFanDuel       82 non-null     float64
 5   FantasyPointsDraftKings    82 non-null     float64
 6   FantasyPointsYahoo         82 non-null     float64
 7   Minutes                    82 non-null     int64  
 8   Seconds                    82 non-null     int64  
 9   Goals                      74 non-null     float64
 10  Assists                    82 non-null     float64
 11  ShotsOnGoal                82 non-null     float64
 12  PowerPlayGoals             82 non-null     float64
 13  ShortHandedGoals           82 non-null     float64
 

map away team `game` to away team `news` and home team `game` to home team `news`

In [15]:
Away_df = pd.merge(df_Game , df_news ,how="inner" , left_on="AwayTeamID" , right_on="TeamID")
Home_df = pd.merge(df_Game , df_news ,how="inner" , left_on="HomeTeamID" , right_on="TeamID")

Determine an `Interval`

In [16]:
pivot_0 = pd.Timedelta(days=0)
pivot_1 = pd.Timedelta(days=7)

In [17]:
mask_Away = (pivot_0 < (Away_df["DateTime_x"] - Away_df["DateTime_y"])) & ((Away_df["DateTime_x"] - Away_df["DateTime_y"]) < pivot_1)
mask_home = (pivot_0 < (Home_df["DateTime_x"] - Home_df["DateTime_y"])) & ((Home_df["DateTime_x"] - Home_df["DateTime_y"]) < pivot_1)

Make DataFrame

In [18]:
final_Away_df = Away_df[mask_Away].sort_values(by="DateTime_x",ascending=False).reset_index()

In [19]:
final_Home_df = Home_df[mask_home].sort_values(by="DateTime_x",ascending=False).reset_index()

Map *News* to `Nearest` *game* 

In [25]:
final_Away_df["DiffrenceInDay"] = final_Away_df["DateTime_x"] - final_Away_df["DateTime_y"]
final_Home_df["DiffrenceInDay"] = final_Home_df["DateTime_x"] - final_Home_df["DateTime_y"]
dup_Away = final_Away_df.sort_values(by="DiffrenceInDay")["NewsID"].duplicated(keep="first")
dup_Home = final_Home_df.sort_values(by="DiffrenceInDay")["NewsID"].duplicated(keep="first")
final_Away_df = final_Away_df[dup_Away.apply(lambda x : not x)]
final_Home_df = final_Home_df[dup_Home.apply(lambda x : not x)]

In [32]:
df_Stat.groupby(by="TeamID").transform("mean")

/tmp/ipykernel_7635/1616597117.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.transform is deprecated. In a future version, a TypeError will be raised. Before calling .transform, select only columns which should be valid for the function.
  df_Stat.groupby(by="TeamID").transform("mean")


,StatID,FantasyPoints,FantasyPointsFanDuel,FantasyPointsDraftKings,FantasyPointsYahoo,Minutes,Seconds,Goals,Assists,ShotsOnGoal,...,GoaltendingMinutes,GoaltendingSeconds,GoaltendingShotsAgainst,GoaltendingGoalsAgainst,GoaltendingSaves,GoaltendingWins,GoaltendingLosses,GoaltendingShutouts,GoaltendingOvertimeLosses,FantasyPointsFantasyDraft
0,2.913655e+06,37.100000,125.350000,105.150000,75.900000,201.000000,3.500000,2.100000,3.150000,24.600000,...,41.500000,26.500000,17.500000,2.500000,14.850000,0.100000,0.000000,0.0,0.450,154.100
1,2.913706e+06,44.700000,159.750000,126.500000,91.600000,203.000000,15.000000,2.500000,4.900000,23.400000,...,41.500000,26.500000,24.000000,1.800000,19.050000,0.550000,0.000000,0.0,0.000,167.800
2,2.914766e+06,21.400000,111.100000,97.050000,43.800000,230.000000,27.000000,1.700000,2.550000,29.750000,...,46.000000,22.500000,21.300000,2.700000,18.100000,0.000000,0.500000,0.0,0.000,143.850
3,2.913470e+06,29.025000,121.825000,104.625000,60.400000,224.750000,22.250000,1.775000,3.075000,23.075000,...,44.500000,23.500000,25.500000,2.275000,23.075000,0.050000,0.525000,0.0,0.000,158.675
4,2.914794e+06,27.400000,124.700000,97.950000,55.800000,237.000000,2.000000,2.300000,4.250000,26.950000,...,47.500000,22.500000,23.150000,3.300000,19.750000,0.000000,0.500000,0.0,0.000,152.500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,2.913707e+06,19.600000,72.833333,57.466667,42.566667,99.666667,1.000000,0.500000,0.966667,12.133333,...,20.000000,9.666667,6.866667,0.500000,5.700000,0.066667,0.066667,0.0,0.000,78.800
78,2.913723e+06,21.566667,67.300000,53.933333,43.733333,100.666667,4.333333,0.500000,0.733333,7.933333,...,20.333333,4.000000,5.600000,0.166667,5.366667,0.133333,0.000000,0.0,0.000,75.700
79,2.914294e+06,3.750000,26.725000,22.950000,9.825000,75.250000,4.500000,0.166667,0.225000,1.950000,...,15.250000,0.250000,5.850000,0.400000,2.175000,0.000000,0.075000,0.0,0.025,41.350
80,2.913776e+06,22.300000,68.900000,55.700000,45.750000,100.000000,0.000000,0.450000,0.700000,6.200000,...,20.000000,0.000000,8.750000,0.200000,8.300000,0.150000,0.000000,0.0,0.000,78.450


## Add Game Stat

In [26]:
final_Away_df = pd.merge(final_Away_df , df_Stat ,how="inner" , left_on="AwayTeamID" , right_on="TeamID")
final_Home_df = pd.merge(final_Home_df , df_Stat ,how="inner" , left_on="HomeTeamID" , right_on="TeamID")

In [27]:
final_Away_df

,index,GameID,DateTime_x,IsClosed,AwayTeamID,HomeTeamID,AwayTeamScore,HomeTeamScore,TeamID_x,NewsID,...,GoaltendingMinutes_y,GoaltendingSeconds_y,GoaltendingShotsAgainst_y,GoaltendingGoalsAgainst_y,GoaltendingSaves_y,GoaltendingWins_y,GoaltendingLosses_y,GoaltendingShutouts_y,GoaltendingOvertimeLosses_y,FantasyPointsFantasyDraft_y
0,4,20858,2023-11-14,True,8,27,1.0,0.0,8,27337,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.7
1,4,20858,2023-11-14,True,8,27,1.0,0.0,8,27337,...,59,0,21.8,2.9,18.8,1.0,0.0,0.0,0.0,211.2
2,4,20858,2023-11-14,True,8,27,1.0,0.0,8,27337,...,63,0,27.5,3.4,24.3,1.1,0.0,0.0,0.0,223.6
3,2,20839,2023-11-11,True,29,21,11.0,8.0,29,27290,...,37,0,22.1,0.9,21.2,0.0,0.0,0.0,0.0,129.5
4,2,20839,2023-11-11,True,29,21,11.0,8.0,29,27290,...,59,42,35.6,3.9,31.6,0.0,0.0,0.0,1.0,184.9
5,2,20839,2023-11-11,True,29,21,11.0,8.0,29,27290,...,24,19,16.7,0.8,14.7,0.2,0.0,0.0,0.0,111.0


In [28]:
final_Home_df

,index,GameID,DateTime_x,IsClosed,AwayTeamID,HomeTeamID,AwayTeamScore,HomeTeamScore,TeamID_x,NewsID,...,GoaltendingMinutes_y,GoaltendingSeconds_y,GoaltendingShotsAgainst_y,GoaltendingGoalsAgainst_y,GoaltendingSaves_y,GoaltendingWins_y,GoaltendingLosses_y,GoaltendingShutouts_y,GoaltendingOvertimeLosses_y,FantasyPointsFantasyDraft_y
0,0,20865,2023-11-16,False,10,14,0.0,0.0,14,27365,...,37,0,20.3,1.7,18.4,0.0,0.0,0.0,0.0,110.6
1,0,20865,2023-11-16,False,10,14,0.0,0.0,14,27365,...,59,0,29.7,2.9,26.7,1.0,0.0,0.0,0.0,252.7
2,0,20865,2023-11-16,False,10,14,0.0,0.0,14,27365,...,24,0,14.3,0.0,14.3,0.2,0.0,0.2,0.0,110.0
3,2,20860,2023-11-15,True,36,25,4.0,5.0,25,27356,...,17,12,2.6,0.2,2.4,0.1,0.0,0.0,0.0,67.2
4,2,20860,2023-11-15,True,36,25,4.0,5.0,25,27356,...,20,0,11.2,0.1,10.9,0.1,0.0,0.0,0.0,74.8
5,2,20860,2023-11-15,True,36,25,4.0,5.0,25,27356,...,24,0,3.0,0.2,2.8,0.2,0.0,0.0,0.0,85.1


## Save data

In [24]:
final_Away_df.to_csv("./Away_data.csv",index=False)
final_Home_df.to_csv("./Home_data.csv",index=False)